In [1]:
# install
#Pkg.add("MNIST");
using MNIST

In [2]:
# training data
X,y = traindata(); 
n = size(X, 2);

In [3]:
inputLayerSize = size(X,1); 
hiddenLayerSize = 50;
outputLayerSize = 10;

In [5]:
# representing each output as an array of size of the output layer
Y = zeros(outputLayerSize, n);
for i = 1:n
    Y[y[i]+1,i] = 1;
end

In [6]:
function sigmoid(z)
    g = 1.0 ./ (1.0 + exp(-z));
    return g;
end

sigmoid (generic function with 1 method)

In [7]:
function sigmoidGradient(z)
  return sigmoid(z).*(1-sigmoid(z));
end

sigmoidGradient (generic function with 1 method)

In [14]:
# weight regularization parameter
lambda = 2; 
function costFunction(truth, prediction)
    #cost = (-truth.*log(prediction)) - ((1-truth).*log(1-prediction));
    cost = (prediction - truth).^2
    # regularization term
    regularization = (lambda/(2*n))*(sum(sum(Theta1[2:end,:].^2)) + sum(sum(Theta2[2:end,:].^2)));

    return (1/n)*sum(sum(cost)) + regularization; # regularized cost
end

costFunction (generic function with 1 method)

In [27]:
# weights with bias
Theta1 = randn(inputLayerSize+1, hiddenLayerSize); 
Theta2 = randn(hiddenLayerSize+1, outputLayerSize); 
# learning rate
alpha = 0.01;
# number of iterations
epoch = 20;
# cost per epoch
J = zeros(epoch,1);
# backpropagation algorithm
for i = 1:epoch
    prediction = 0;
    for j = 1:n # for each input
        # Feedforward #
        a1 = [1, X[:,j]]; # add one bias element
        z2 = Theta1'*a1;
        a2 = sigmoid(z2);
        a2 = [1, a2]; # add one bias element
        z3 = Theta2'*a2;
        a3 = sigmoid(z3);

        if j==n
            prediction = a3;
        end
        
        # Backpropagation process #
        delta3 = (a3 - Y[:,j]);
        delta2 = (Theta2[2:end,:]*delta3).*sigmoidGradient(z2) ;
        
        #update weights
        Theta2 = Theta2 - alpha* a2*delta3';
        Theta1 = Theta1 - alpha* a1*delta2';
        
    end
    J[i,:] = costFunction(Y, prediction);
end

In [28]:
J

20x1 Array{Float64,2}:
 4.43235
 4.54629
 4.50096
 5.37763
 5.19999
 5.27255
 4.46339
 5.42728
 4.84521
 4.91514
 4.78945
 4.54475
 5.12584
 5.68911
 4.77508
 5.48915
 6.08045
 6.32731
 5.73911
 4.96492

In [18]:
function predict(Theta1Wt, Theta2Wt, data)
    dataSize = size(data, 2); 
    p = zeros(dataSize, 1);
    h1 = sigmoid(Theta1Wt'*[ones(1,size(data,2)), data]);
    h2 = sigmoid(Theta2Wt'*[ones(1,size(h1,2)), h1]);
    # 1 index is for 0, 2 for 1 ...so forth
    for i=1:dataSize
        p[i,:] = indmax(h2[:,i])-1;
    end
    return p;
end

function accuracy(truth, prediction)
    dataSize = length(truth);
    match =0;
    for i=1:dataSize
        if truth[i,:] == pred[i,:]
            match = match +1;
        end
    end
    return (match/dataSize)*100;
end

accuracy (generic function with 1 method)

In [23]:
pred = predict(Theta1, Theta2, X);
println("train accuracy: ", accuracy(y, pred));

train accuracy: 59.92666666666666


In [1]:
using PyPlot

Qt: Untested Windows version 10.0 detected!
INFO: Loading help data...


Figure(PyObject <matplotlib.figure.Figure object at 0x0000000034853908>)